In [46]:
import pandas as pd
from sklearn import model_selection
import matplotlib.pyplot as plt
#For enabling intellisense
%config IPCompleter.greedy=True

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
df = pd.read_csv('train.txt')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [6]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [9]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,Urban,Y
4,LP001008,Male,No,0,Graduate,No,Urban,Y


In [18]:
obj_df["Property_Area"].value_counts()

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [24]:
cleanup_nums = {"Gender":  {"Male": 0, "Female": 1},
                "Married": {"Yes": 1, "No": 0},
                "Dependents":{"0":0,"1":1,"2":2,"3+":3},
                "Education":{"Graduate":1, "Not Graduate":0},
                "Self_Employed":{"Yes":1,"No":0},
                "Property_Area":{"Urban":1,"Semiurban":0,"Rural":2},
                "Loan_Status": {"Y":1, "N":0}}

In [26]:
#obj_df.replace(cleanup_nums, inplace=True)
obj_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,0.0,0.0,0.0,1,0.0,1,1
1,LP001003,0.0,1.0,1.0,1,0.0,2,0
2,LP001005,0.0,1.0,0.0,1,1.0,1,1
3,LP001006,0.0,1.0,0.0,0,0.0,1,1
4,LP001008,0.0,0.0,0.0,1,0.0,1,1


In [27]:
#df.replace(cleanup_nums, inplace=True)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0.0,0.0,0.0,1,0.0,5849,0.0,NaN,360.0,1.0,1,1
1,LP001003,0.0,1.0,1.0,1,0.0,4583,1508.0,128.0,360.0,1.0,2,0
2,LP001005,0.0,1.0,0.0,1,1.0,3000,0.0,66.0,360.0,1.0,1,1
3,LP001006,0.0,1.0,0.0,0,0.0,2583,2358.0,120.0,360.0,1.0,1,1
4,LP001008,0.0,0.0,0.0,1,0.0,6000,0.0,141.0,360.0,1.0,1,1


In [28]:
df.shape

(614, 13)

In [29]:
df.dropna(how='any').shape

(480, 13)

In [31]:
df.dropna(how='any', inplace=True)

In [109]:
test = pd.read_csv('test.txt')
test.shape

(367, 12)

In [107]:
df.head()
df.shape

(480, 13)

In [129]:
test = pd.read_csv('test.txt')

test.replace(cleanup_nums, inplace=True)
test.dropna(how='any', inplace=True)
array = test.values
TestX = array[:,1:12]

In [49]:
# Split-out validation dataset
array = df.values
X = array[:,1:12]
Y = array[:,12]
Y=Y.astype('int')
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [50]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

In [52]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.812483 (0.053261)
LDA: 0.817746 (0.055865)
KNN: 0.666869 (0.083656)
CART: 0.702901 (0.087622)
NB: 0.804656 (0.062010)
SVM: 0.711471 (0.061768)


In [123]:
lda = LinearDiscriminantAnalysis(solver='svd', shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001)
lda.fit(X_train, Y_train)
predictions = lda.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.7708333333333334
[[14 21]
 [ 1 60]]
              precision    recall  f1-score   support

           0       0.93      0.40      0.56        35
           1       0.74      0.98      0.85        61

   micro avg       0.77      0.77      0.77        96
   macro avg       0.84      0.69      0.70        96
weighted avg       0.81      0.77      0.74        96



In [131]:
predictionsX = lda.predict(TestX)
print(len(predictionsX))

submission = pd.DataFrame({'Loan_Id':test['Loan_ID'],'Loan_Status':predictionsX})

#Visualize the first 5 rows
submission.head()

289


,Loan_Id,Loan_Status
0,LP001015,1
1,LP001022,1
2,LP001031,1
4,LP001051,1
5,LP001054,1


In [132]:
filename = 'Predictions.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Predictions.csv


In [83]:
lr = LogisticRegression(solver='liblinear', multi_class='ovr')
lr.fit(X_train, Y_train)
predictions = lr.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.7708333333333334
[[15 20]
 [ 2 59]]
              precision    recall  f1-score   support

           0       0.88      0.43      0.58        35
           1       0.75      0.97      0.84        61

   micro avg       0.77      0.77      0.77        96
   macro avg       0.81      0.70      0.71        96
weighted avg       0.80      0.77      0.75        96



In [57]:
gb = GaussianNB()
gb.fit(X_train, Y_train)
predictions = gb.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.7395833333333334
[[15 20]
 [ 5 56]]
              precision    recall  f1-score   support

           0       0.75      0.43      0.55        35
           1       0.74      0.92      0.82        61

   micro avg       0.74      0.74      0.74        96
   macro avg       0.74      0.67      0.68        96
weighted avg       0.74      0.74      0.72        96

